# CatBoost Starter for Brain Comp
EEGの差分の集計特徴量を用いたNotebook。

- ver1: CV = 1.26,
- ver2: CV = 1.175, LB = 0.93
- ver3: CV = 1.168, LB = 0.98
- ver4: CV = 0.798, LB = 0.63

### Version Notes
- Version 1 - EEGのT=20, 30, 40秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。また、データの1/20のみ利用。
- Version 2 - EEGのT=20, 30秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。時間窓に必要な部分のみ計算することで高速化した。
- Version 3 - EEGのT=10, 20, 25, 30, 40秒における、5秒時間窓 / 10秒時間窓のmean, min, max, std, max-minを特徴量とした。差分をとる電極を追加した。
- Version 4 - 公開notebookの、スペクトログラムによる特徴量を追加した。

# Load Libraries

In [35]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import time
pl.Config.set_tbl_cols(-1)

VER = 4

# Load Test Data

In [36]:
df_test = pl.read_csv('../input/hms-harmful-brain-activity-classification/test.csv')
print('Train shape:', df_test.shape )
df_test.head()

Train shape: (1, 3)


spectrogram_id,eeg_id,patient_id
i64,i64,i64
853520,3911565283,6885


# Create Multiple Eeg Id Train Data
データの説明[here][1]には、テストデータには、同じ `eeg_id` からの複数の crop は含まれていないと記載されている。

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

# Feature Engineer


CatBoostモデルに入れる特徴量を作成する。


## 1. spectrogram
KaggleスペクトログラムとEEGスペクトログラムの両方を使用する特徴量

In [37]:
test = df_test.to_pandas()

In [38]:
%time
# ENGINEER FEATURES

import warnings
warnings.filterwarnings('ignore')
PATH = '../input/hms-harmful-brain-activity-classification/train_spectrograms/'
# FEATURE NAMES
# 元の Spectrogram のファイルから、列名を取得
SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]
FEATURES = [f'{c}_mean_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間平均
FEATURES += [f'{c}_min_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間最小値
FEATURES += [f'{c}_mean_20s' for c in SPEC_COLS]            # スペクトログラム: 20秒の時間窓の時間平均
FEATURES += [f'{c}_min_20s' for c in SPEC_COLS]             # スペクトログラム: 20秒の時間窓の時間最小値
FEATURES += [f'eeg_mean_f{x}_10s' for x in range(512)]      # 脳波: 10分の時間窓の時間平均
FEATURES += [f'eeg_min_f{x}_10s' for x in range(512)]       # 脳波: 10分の時間窓の時間最小値
FEATURES += [f'eeg_max_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均
FEATURES += [f'eeg_std_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均

CPU times: total: 0 ns
Wall time: 0 ns


In [39]:
import pywt, librosa

USE_WAVELET = None 

NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

# DENOISE FUNCTION
# dの平均からの絶対偏差の平均を返す
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

# wavelet関数を使って雑音のしきい値を計算し、それよりも小さい周波数成分を除去する
def denoise(x, wavelet='haar', level=1):    
    # xにウェーブレット変換(DWT)し、係数を計算する
    coeff = pywt.wavedec(x, wavelet, mode="per")
    # 係数の最後のレベル（詳細係数）からノイズの標準偏差を計算
    sigma = (1/0.6745) * maddest(coeff[-level])

    # ノイズのしきい値を計算
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    # しきい値処理し、ノイズを除去
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:]) # 

    # 修正された係数を使用して逆ウェーブレット変換(IDWT)を施し、ノイズ除去された信号を返す
    ret=pywt.waverec(coeff, wavelet, mode='per') 
    
    return ret


# EEG信号からスペクトログラムを計算する関数
def spectrogram_from_eeg(parquet_path, display=False):
    # EEGデータは4つのモンタージュに分かれている。
    # 各モンタージュのスペクトログラムを計算して、4つのスペクトログラムを結合した3次元配列として返す
    
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]
    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((128,256,4),dtype='float32')
    
    if display: plt.figure(figsize=(10,7))
    signals = []
    # EEG信号は4つのモンタージュに分かれているため、スペクトログラムを作成するモンタージュを選択
    for k in range(4):
        COLS = FEATS[k]
        
        # 差分をとるモンタージュを選択
        for kk in range(4):
        
            # COMPUTE PAIR DIFFERENCES: モンタージュ信号間の差分をとる
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
            else: x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM: 差分信号に対してメル周波数スペクトログラムを計算する
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//256, 
                  n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

            # LOG TRANSFORM: ログ変換
            width = (mel_spec.shape[1]//32)*32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

            # STANDARDIZE TO -1 TO 1: 標準化
            mel_spec_db = (mel_spec_db+40)/40 
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        # 各モンタージュ間の差分の平均をとる
        img[:,:,k] /= 4.0
        
        if display:
            plt.subplot(2,2,k+1)
            plt.imshow(img[:,:,k],aspect='auto',origin='lower')
            plt.title(f'EEG {eeg_id} - Spectrogram {NAMES[k]}')
            
    if display: 
        plt.show()
        plt.figure(figsize=(10,5))
        offset = 0
        for k in range(4):
            if k>0: offset -= signals[3-k].min()
            plt.plot(range(10_000),signals[k]+offset,label=NAMES[3-k])
            offset += signals[3-k].max()
        plt.legend()
        plt.title(f'EEG {eeg_id} Signals')
        plt.show()
        print(); print('#'*25); print()
        
    return img

In [40]:
# CREATE ALL EEG SPECTROGRAMS
# テストデータのeegを用いてスペクトログラムを計算
PATH2 = '../input/hms-harmful-brain-activity-classification/test_eegs/'
DISPLAY = 0
EEG_IDS2 = test.eeg_id.unique()
all_eegs2 = {}

print('Converting Test EEG to Spectrograms...'); print()
for i,eeg_id in enumerate(EEG_IDS2):
        
    # CREATE SPECTROGRAM FROM EEG PARQUET
    img = spectrogram_from_eeg(f'{PATH2}{eeg_id}.parquet', i<DISPLAY)
    all_eegs2[eeg_id] = img

Converting Test EEG to Spectrograms...



In [41]:
# FEATURE ENGINEER TEST
# trainデータと同じ特徴量の作成

PATH2 = '../input/hms-harmful-brain-activity-classification/test_spectrograms/'
data = np.zeros((len(test),len(FEATURES)))
    
for k in range(len(test)):
    row = test.iloc[k]
    s = int( row.spectrogram_id )
    spec = pd.read_parquet(f'{PATH2}{s}.parquet')
    
    # 10 MINUTE WINDOW FEATURES
    x = np.nanmean( spec.iloc[:,1:].values, axis=0)
    data[k,:400] = x
    x = np.nanmin( spec.iloc[:,1:].values, axis=0)
    data[k,400:800] = x

    # 20 SECOND WINDOW FEATURES
    x = np.nanmean( spec.iloc[145:155,1:].values, axis=0)
    data[k,800:1200] = x
    x = np.nanmin( spec.iloc[145:155,1:].values, axis=0)
    data[k,1200:1600] = x
    
    # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
    eeg_spec = np.zeros((512,256),dtype='float32')
    xx = all_eegs2[row.eeg_id]
    for j in range(4): eeg_spec[128*j:128*(j+1),] = xx[:,:,j]

    # 10 SECOND WINDOW FROM EEG SPECTROGRAMS 
    x = np.nanmean(eeg_spec.T[100:-100,:],axis=0)
    data[k,1600:2112] = x
    x = np.nanmin(eeg_spec.T[100:-100,:],axis=0)
    data[k,2112:2624] = x
    x = np.nanmax(eeg_spec.T[100:-100,:],axis=0)
    data[k,2624:3136] = x
    x = np.nanstd(eeg_spec.T[100:-100,:],axis=0)
    data[k,3136:3648] = x

test[FEATURES] = data
print('New test shape',test.shape)

New test shape (1, 3651)


In [42]:
test.head()

,spectrogram_id,eeg_id,patient_id,LL_0.59_mean_10m,LL_0.78_mean_10m,LL_0.98_mean_10m,LL_1.17_mean_10m,LL_1.37_mean_10m,LL_1.56_mean_10m,LL_1.76_mean_10m,...,eeg_std_f502_10s,eeg_std_f503_10s,eeg_std_f504_10s,eeg_std_f505_10s,eeg_std_f506_10s,eeg_std_f507_10s,eeg_std_f508_10s,eeg_std_f509_10s,eeg_std_f510_10s,eeg_std_f511_10s
0,853520,3911565283,6885,16.864132,19.120565,18.342468,13.408634,8.0575,4.890133,3.460633,...,0.085247,0.090794,0.091528,0.08319,0.078124,0.07424,0.072341,0.07337,0.079924,0.076502


In [43]:
test_spec = test.drop(["spectrogram_id", "patient_id"], axis = 1)

In [44]:
del test
gc.collect()

0

## 2. 生のEEG
生のEEGの集計特徴量

In [45]:
# parameter
# 差分特徴量の集計特徴量をつくる
# window = 4000 # 20秒窓 (20秒 / (1レコード1/200秒) = 4000レコード)
center_seconds_list = [10, 10, 20, 20, 25, 25, 30, 30, 40, 40]
window_list         = [1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000]

In [46]:
def create_aggregated_features_on_EEG_differences(df_eeg_diff, window, center_sec):
    """
    @Args
    df_eeg_diff : 差分特徴量を持つデータフレーム
    window      : 時間窓として抽出する行数 [行]
    center_sec  : 時間窓の中心時刻 [s]
    
    @Returns
    df_features    : 集計特徴量を持つデータフレーム
    """
    ### 時間窓の設定
    window_time = window / (200)
    str_window = str(int(window_time)) + "s"

    # 時間窓 [window_min, window_max] に入るレコードのみ抽出
    window_max = center_sec + (window / 2)
    window_min = center_sec - (window / 2)

    df_eeg_diff = df_eeg_diff.filter((pl.col("seconds_eeg") <= window_max) & (pl.col("seconds_eeg") >= window_min))
    df_eeg_diff = df_eeg_diff.drop(['seconds_eeg'])

    
    ### 特徴量作成
    
    df_features = df_eeg_diff.group_by(by = "eeg_id").agg(
        ### 簡易特徴量
        # LL: Fp1 - T3
        pl.col("LL_Fp1-T3").mean().alias("mean_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").max().alias("min_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").min().alias("max_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").std().alias("std_" + str_window + "_LL_Fp1-T3"),
        (pl.col("LL_Fp1-T3").max() - pl.col("LL_Fp1-T3").min()).alias("max-min_" + str_window + "_LL_Fp1-T3"),

        # LL: Fp1 - T3
        pl.col("LL_T3-O1").mean().alias("mean_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").max().alias("min_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").min().alias("max_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").std().alias("std_" + str_window + "_LL_T3-O1"),
        (pl.col("LL_T3-O1").max() - pl.col("LL_T3-O1").min()).alias("max-min__" + str_window + "_LL_T3-O1"),


        # LP: Fp1 - C3
        pl.col("LP_Fp1-C3").mean().alias("mean_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").max().alias("min_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").min().alias("max_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").std().alias("std_" + str_window + "_LP_Fp1-C3"),
        (pl.col("LP_Fp1-C3").max() - pl.col("LP_Fp1-C3").min()).alias("max-min_" + str_window + "_LP_Fp1-C3"),

        # LP: C3 - O1
        pl.col("LP_C3-O1").mean().alias("mean_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").max().alias("min_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").min().alias("max_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").std().alias("std_" + str_window + "_LP_C3-O1"),
        (pl.col("LP_C3-O1").max() - pl.col("LP_C3-O1").min()).alias("max-min_" + str_window + "_LP_C3-O1"),


        # RP: Fp1 - C4
        pl.col("RP_Fp2-C4").mean().alias("mean_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").max().alias("min_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").min().alias("max_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").std().alias("std_" + str_window + "_RP_Fp2-C4"),
        (pl.col("RP_Fp2-C4").max() - pl.col("RP_Fp2-C4").min()).alias("max-min_" + str_window + "_RP_Fp2-C4"),

        # RP: C4 - O2
        pl.col("RP_C4-O2").mean().alias("mean_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").max().alias("min_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").min().alias("max_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").std().alias("std_" + str_window + "_RP_C4-O2"),
        (pl.col("RP_C4-O2").max() - pl.col("RP_C4-O2").min()).alias("max-min_" + str_window + "_RP_C4-O2"),


        # RR: Fp2 - T4
        pl.col("RR_Fp2-T4").mean().alias("mean_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").max().alias("min_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").min().alias("max_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").std().alias("std_" + str_window + "_RR_Fp2-T4"),
        (pl.col("RR_Fp2-T4").max() - pl.col("RR_Fp2-T4").min()).alias("max-min_" + str_window + "_RR_Fp2-T4"),

        # RR: T4 - O2
        pl.col("RR_T4-O2").mean().alias("mean_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").max().alias("min_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").min().alias("max_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").std().alias("std_" + str_window + "_RR_T4-O2"),
        (pl.col("RR_T4-O2").max() - pl.col("RR_T4-O2").min()).alias("max-min_" + str_window + "_RR_T4-O2"),

        
        ### 全差分の特徴量
        # LL: Fp1 - F7
        pl.col("LL_Fp1-F7").mean().alias("mean_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").max().alias("min_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").min().alias("max_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").std().alias("std_" + str_window + "_LL_Fp1-F7"),
        (pl.col("LL_Fp1-F7").max() - pl.col("LL_Fp1-F7").min()).alias("max-min_" + str_window + "_LL_Fp1-F7"),

        # LL: F7 - T3
        pl.col("LL_F7-T3").mean().alias("mean_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").max().alias("min_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").min().alias("max_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").std().alias("std_" + str_window + "_LL_F7-T3"),
        (pl.col("LL_F7-T3").max() - pl.col("LL_F7-T3").min()).alias("max-min_" + str_window + "_LL_F7-T3"),

        # LL: T3 - T5
        pl.col("LL_T3-T5").mean().alias("mean_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").max().alias("min_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").min().alias("max_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").std().alias("std_" + str_window + "_LL_T3-T5"),
        (pl.col("LL_T3-T5").max() - pl.col("LL_T3-T5").min()).alias("max-min_" + str_window + "_LL_T3-T5"),

        # LL: T5 - O1
        pl.col("LL_T5-O1").mean().alias("mean_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").max().alias("min_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").min().alias("max_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").std().alias("std_" + str_window + "_LL_T5-O1"),
        (pl.col("LL_T5-O1").max() - pl.col("LL_T5-O1").min()).alias("max-min_" + str_window + "_LL_T5-O1"),



        # LP: Fp1 - F3
        pl.col("LP_Fp1-F3").mean().alias("mean_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").max().alias("min_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").min().alias("max_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").std().alias("std_" + str_window + "_LP_Fp1-F3"),
        (pl.col("LP_Fp1-F3").max() - pl.col("LP_Fp1-F3").min()).alias("max-min_" + str_window + "_LP_Fp1-F3"),

        # LP: F3 - C3
        pl.col("LP_F3-C3").mean().alias("mean_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").max().alias("min_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").min().alias("max_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").std().alias("std_" + str_window + "_LP_F3-C3"),
        (pl.col("LP_F3-C3").max() - pl.col("LP_F3-C3").min()).alias("max-min_" + str_window + "_LP_F3-C3"),

        # LP: C3 - P3
        pl.col("LP_C3-P3").mean().alias("mean_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").max().alias("min_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").min().alias("max_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").std().alias("std_" + str_window + "_LP_C3-P3"),
        (pl.col("LP_C3-P3").max() - pl.col("LP_C3-P3").min()).alias("max-min_" + str_window + "_LP_C3-P3"),

        # LP: P3 - O1
        pl.col("LP_P3-O1").mean().alias("mean_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").max().alias("min_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").min().alias("max_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").std().alias("std_" + str_window + "_LP_P3-O1"),
        (pl.col("LP_P3-O1").max() - pl.col("LP_P3-O1").min()).alias("max-min_" + str_window + "_LP_P3-O1"),



        # RP: Fp2 - F4
        pl.col("RP_Fp2-F4").mean().alias("mean_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").max().alias("min_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").min().alias("max_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").std().alias("std_" + str_window + "_RP_Fp2-F4"),
        (pl.col("RP_Fp2-F4").max() - pl.col("RP_Fp2-F4").min()).alias("max-min_" + str_window + "_RP_Fp2-F4"),

        # RP: F4 - C4
        pl.col("RP_F4-C4").mean().alias("mean_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").max().alias("min_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").min().alias("max_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").std().alias("std_" + str_window + "_RP_F4-C4"),
        (pl.col("RP_F4-C4").max() - pl.col("RP_F4-C4").min()).alias("max-min_" + str_window + "_RP_F4-C4"),

        # RP: C4 - P4
        pl.col("RP_C4-P4").mean().alias("mean_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").max().alias("min_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").min().alias("max_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").std().alias("std_" + str_window + "_RP_C4-P4"),
        (pl.col("RP_C4-P4").max() - pl.col("RP_C4-P4").min()).alias("max-min_" + str_window + "_RP_C4-P4"),

        # RP: P4 - O2
        pl.col("RP_P4-O2").mean().alias("mean_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").max().alias("min_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").min().alias("max_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").std().alias("std_" + str_window + "_RP_P4-O2"),
        (pl.col("RP_P4-O2").max() - pl.col("RP_P4-O2").min()).alias("max-min_" + str_window + "_RP_P4-O2"),


        
        # RR: Fp2 - F8
        pl.col("RR_Fp2-F8").mean().alias("mean_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").max().alias("min_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").min().alias("max_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").std().alias("std_" + str_window + "_RR_Fp2-F8"),
        (pl.col("RR_Fp2-F8").max() - pl.col("RR_Fp2-F8").min()).alias("max-min_" + str_window + "_RR_Fp2-F8"),

        # RR: F8 - T4
        pl.col("RR_F8-T4").mean().alias("mean_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").max().alias("min_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").min().alias("max_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").std().alias("std_" + str_window + "_RR_F8-T4"),
        (pl.col("RR_F8-T4").max() - pl.col("RR_F8-T4").min()).alias("max-min_" + str_window + "_RR_F8-T4"),

        # RR: T4 - T6
        pl.col("RR_T4-T6").mean().alias("mean_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").max().alias("min_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").min().alias("max_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").std().alias("std_" + str_window + "_RR_T4-T6"),
        (pl.col("RR_T4-T6").max() - pl.col("RR_T4-T6").min()).alias("max-min_" + str_window + "_RR_T4-T6"),

        # RR: T6 - O2
        pl.col("RR_T6-O2").mean().alias("mean_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").max().alias("min_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").min().alias("max_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").std().alias("std_" + str_window + "_RR_T6-O2"),
        (pl.col("RR_T6-O2").max() - pl.col("RR_T6-O2").min()).alias("max-min_" + str_window + "_RR_T6-O2"),        
    )

    
    # 差分特徴量をdrop
    df_features = df_features.drop(
        ['LL_Fp1-T3', 'LL_T3-O1', 'LP_Fp1-C3', 'LP_C3-O1', 'RP_Fp2-C4', 'RP_C4-O2', 'RR_Fp2-T4', 'RR_T4-O2',
         'LL_Fp1-F7', 'LL_F7-T3', 'LL_T3-T5', 'LL_T5-O1',
         'LP_Fp1-F3', 'LP_F3-C3', 'LP_C3-P3', 'LP_P3-O1',
         'RP_Fp2-F4', 'RP_F4-C4', 'RP_C4-P4', 'RP_P4-O2',
         'RR_Fp2-F8', 'RR_F8-T4', 'RR_T4-T6', 'RR_T6-O2',]
        )
    
    
    # 列名を一括で変更
    col_suffix = "_at_" + str(center_sec) + "s"
    list_original_col = df_features.columns
    df_features = df_features.with_columns(pl.all().name.suffix(col_suffix)).drop(list_original_col) # suffixを付与して、元の列名の列を削除
    
    # eeg_id 列の列名を戻す
    eeg_col = "eeg_id_at_" + str(center_sec) + "s"
    df_features = df_features.rename({eeg_col : "eeg_id"})


    return df_features

In [47]:
%%time
PATH = '../input/hms-harmful-brain-activity-classification/test_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

start = time.time()
df_all_diff_features = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0:
        print(i, ', ', end = '')
        
        ### 状況確認
        # 時間計測
        end = time.time()
        seconds = end - start
        seconds = int(seconds + 0.5)    # 秒数を四捨五入
        h = seconds // 3600             # 時の取得
        m = (seconds - h * 3600) // 60  # 分の取得
        s = seconds - h * 3600 - m * 60 # 秒の取得
        print(f"{h:02}:{m:02}:{s:02}" )
        start = time.time()

        # 形状確認
        print(df_all_diff_features.shape)

    # if i%10 != 0: continue

    eeg_id = int(f.split('.')[0])    

    # 読み込み
    temp_eeg = pl.read_parquet(f'{PATH}{f}')

    # eeg_id を追加
    temp_eeg = temp_eeg.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

    # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
    t = 1/200 # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す
    temp_eeg = temp_eeg.with_row_count("id")
    temp_eeg = temp_eeg.with_columns((pl.col("id") * t).alias("seconds_eeg"))   # 計測時刻の列をつくる
    temp_eeg = temp_eeg.filter(pl.col("seconds_eeg") < 50).drop(["id"])         # 50秒以降のデータは使わない
    
    # 差分特徴量を作る
    temp_eeg = temp_eeg.with_columns(
        # 簡易特徴量
        (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
        (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
        (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
        (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
        (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
        (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
        (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
        (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),

        # 全差分の特徴量
        (pl.col("Fp1") - pl.col("F7")).alias("LL_Fp1-F7"),
        (pl.col("F7") - pl.col("T3")).alias("LL_F7-T3"),
        (pl.col("T3") - pl.col("T5")).alias("LL_T3-T5"),
        (pl.col("T5") - pl.col("O1")).alias("LL_T5-O1"),

        (pl.col("Fp1") - pl.col("F3")).alias("LP_Fp1-F3"),
        (pl.col("F3") - pl.col("C3")).alias("LP_F3-C3"),
        (pl.col("C3") - pl.col("P3")).alias("LP_C3-P3"),
        (pl.col("P3") - pl.col("O1")).alias("LP_P3-O1"),

        (pl.col("Fp2") - pl.col("F4")).alias("RP_Fp2-F4"),
        (pl.col("F4") - pl.col("C4")).alias("RP_F4-C4"),
        (pl.col("C4") - pl.col("P4")).alias("RP_C4-P4"),
        (pl.col("P4") - pl.col("O2")).alias("RP_P4-O2"),

        (pl.col("Fp2") - pl.col("F8")).alias("RR_Fp2-F8"),
        (pl.col("F8") - pl.col("T4")).alias("RR_F8-T4"),
        (pl.col("T4") - pl.col("T6")).alias("RR_T4-T6"),
        (pl.col("T6") - pl.col("O2")).alias("RR_T6-O2"),
        
    )

    # 差分特徴量以外drop
    temp_eeg = temp_eeg.drop(['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG'])
    
    # eeg_id のみ持つデータフレームを作成
    df_eeg_feature = pl.DataFrame({"eeg_id": str(eeg_id)})

    # ひとつのeeg_id のデータフレームに対して特徴量を作成
    for center_sec, window in zip(center_seconds_list, window_list):
            df_features = create_aggregated_features_on_EEG_differences(temp_eeg, window, center_sec)
            df_eeg_feature = df_eeg_feature.join(df_features, on = "eeg_id", how = "left")

    # 全ての eeg_id に対して結合
    df_all_diff_features = pl.concat([df_all_diff_features, df_eeg_feature], how = 'vertical')

There are 1 eeg parquet files
0 , 00:00:00
(0, 0)
CPU times: total: 15.6 ms
Wall time: 40.2 ms


In [48]:
df_all_diff_features.head()

eeg_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_RR_T4-O2_at_10s,std_10s_RR_T4-O2_at_10s,max-min_10s_RR_T4-O2_at_10s,mean_10s_LL_Fp1-F7_at_10s,min_10s_LL_Fp1-F7_at_10s,

In [49]:
df_all_diff_features = df_all_diff_features.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
df_all_diff_features.tail(5)

eeg_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_RR_T4-O2_at_10s,std_10s_RR_T4-O2_at_10s,max-min_10s_RR_T4-O2_at_10s,mean_10s_LL_Fp1-F7_at_10s,min_10s_LL_Fp1-F7_at_10s,

## 3. 各特徴量をjoin

In [50]:
# 集計特徴量をjoinc
df_test = df_test.join(df_all_diff_features, on = "eeg_id", how = "left")
df_test.head()

spectrogram_id,eeg_id,patient_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_RR_T4-O2_at_10s,std_10s_RR_T4-O2_at_10s,max-min_10s_RR_T4-O2_at_10s,mean_10s_LL_Fp1-F7_at_10s

In [51]:
# spec特徴量をjoin
test_spec = pl.from_pandas(test_spec)
df_test = df_test.join(test_spec, on = "eeg_id", how = "left")

## 4. CatBoostに入れるための準備

In [52]:
# Testデータでは、eeg_id, patient_id, spectrogram_id, label_idは1つしかない
FEATURES = df_test.drop(["eeg_id", "spectrogram_id", "patient_id"]).columns
FEATURES

['mean_5s_LL_Fp1-T3_at_10s',
 'min_5s_LL_Fp1-T3_at_10s',
 'max_5s_LL_Fp1-T3_at_10s',
 'std_5s_LL_Fp1-T3_at_10s',
 'max-min_5s_LL_Fp1-T3_at_10s',
 'mean_5s_LL_T3-O1_at_10s',
 'min_5s_LL_T3-O1_at_10s',
 'max_5s_LL_T3-O1_at_10s',
 'std_5s_LL_T3-O1_at_10s',
 'max-min__5s_LL_T3-O1_at_10s',
 'mean_5s_LP_Fp1-C3_at_10s',
 'min_5s_LP_Fp1-C3_at_10s',
 'max_5s_LP_Fp1-C3_at_10s',
 'std_5s_LP_Fp1-C3_at_10s',
 'max-min_5s_LP_Fp1-C3_at_10s',
 'mean_5s_LP_C3-O1_at_10s',
 'min_5s_LP_C3-O1_at_10s',
 'max_5s_LP_C3-O1_at_10s',
 'std_5s_LP_C3-O1_at_10s',
 'max-min_5s_LP_C3-O1_at_10s',
 'mean_5s_RP_Fp2-C4_at_10s',
 'min_5s_RP_Fp2-C4_at_10s',
 'max_5s_RP_Fp2-C4_at_10s',
 'std_5s_RP_Fp2-C4_at_10s',
 'max-min_5s_RP_Fp2-C4_at_10s',
 'mean_5s_RP_C4-O2_at_10s',
 'min_5s_RP_C4-O2_at_10s',
 'max_5s_RP_C4-O2_at_10s',
 'std_5s_RP_C4-O2_at_10s',
 'max-min_5s_RP_C4-O2_at_10s',
 'mean_5s_RR_Fp2-T4_at_10s',
 'min_5s_RR_Fp2-T4_at_10s',
 'max_5s_RR_Fp2-T4_at_10s',
 'std_5s_RR_Fp2-T4_at_10s',
 'max-min_5s_RR_Fp2-T4_at_10s',

In [53]:
df_test[FEATURES].shape

(1, 4848)

In [54]:
# pandasに変換
df_test = df_test.to_pandas()

In [55]:
df_test.head()

,spectrogram_id,eeg_id,patient_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,...,eeg_std_f502_10s,eeg_std_f503_10s,eeg_std_f504_10s,eeg_std_f505_10s,eeg_std_f506_10s,eeg_std_f507_10s,eeg_std_f508_10s,eeg_std_f509_10s,eeg_std_f510_10s,eeg_std_f511_10s
0,853520,3911565283,6885,-2.701218,114.459999,-124.349998,26.186747,238.809998,-12.949674,221.73999,...,0.085247,0.090794,0.091528,0.08319,0.078124,0.07424,0.072341,0.07337,0.079924,0.076502


In [56]:
df_test[FEATURES].shape

(1, 4848)

# Train CatBoost
CatBoost は、超高速トレーニングのために (パラメーター `task_type='GPU'` を追加すると) 両方の Kaggle T4 GPU を自動的に使用する。

In [57]:
import catboost as cat
from catboost import CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

CatBoost version 1.2.2


## Create Model Folder

In [58]:
%%time
from sklearn.model_selection import KFold, GroupKFold
import json

preds = []

for i in range(5):
    print(i,', ',end='')
    model = CatBoostClassifier(task_type='GPU')
    model.load_model(f'../input/catboost-eeg-feature-ver{VER}-train/models/CAT_v{VER}_f{i}.cat')
    
    test_pool = Pool(
        data = df_test[FEATURES]
    )
    
    pred = model.predict_proba(test_pool)
    preds.append(pred)
pred = np.mean(preds,axis=0)
print()
print('Test preds shape',pred.shape)


0 , 1 , 2 , 3 , 4 , 
Test preds shape (1, 6)
CPU times: total: 750 ms
Wall time: 983 ms


In [59]:
sub = pd.DataFrame({'eeg_id':df_test.eeg_id.values})
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

Submissionn shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.093603,0.069795,0.016764,0.061074,0.190036,0.568728


In [60]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float64